### Импорт данных в БД с Pandas

Этот ноутбук показывает, как импортировать данные в Pandas и подготовить их для загрузки в локальную базу данных SQLite.  
Здесь демонстрируются примеры чтения CSV-файлов, первичной очистки и подготовки таблиц перед записью в БД.

SQLite — это лёгкая встроенная система управления базами данных, которая хранит данные в одном файле .db.  
Такая БД удобна для локальных проектов, прототипирования аналитики и выполнения SQL-запросов без необходимости поднимать отдельный сервер.


In [12]:
import pandas as pd
import numpy as np
import sqlite3

from pathlib import Path

In [13]:
# загрузка данных
categories_df = pd.read_csv('data/categories.csv')
clients_df = pd.read_csv('data/clients.csv')
subscriptions_df = pd.read_csv('data/subscriptions.csv')
transactions_df = pd.read_csv('data/transactions.csv')

In [14]:
transactions_df.rename(columns={'Unnamed: 0' : 'id'}, inplace=True)

In [23]:
subscriptions_df.head()

,id,client_id,product_category,product_company,amount,date_start,date_end
0,766278,825,29,NaN,3990,2012-01-25,NaT
1,658393,709,4,Яндекс.Музыка,199,2012-01-27,2020-02-27
2,354548,383,4,Boom,149,2012-01-28,NaT
3,515830,556,4,Spotify,169,2012-02-01,NaT
4,799560,862,4,YouTube Music,169,2012-02-20,2020-05-20


In [24]:
clients_df.head()

,id,fullname,address,phone_number,email,workplace,birthdate,registration_date,gender,income,expenses,credit,deposit
0,1,Волкова Фаина Афанасьевна,"с. Новосибирск, наб. Привокзальная, д. 58 к. 5...",8 (475) 014-6494,wkirillov@rambler.ru,"ст. Кировск (Мурм.), ш. Коммуны, д. 238 стр. 3...",1966-09-07,2017-01-10,F,230910.0,61975,1.0,NaN
1,2,Пелагея Вячеславовна Галкина,"клх Белокуриха, ш. Комсомольское, д. 701 стр. ...",+7 (981) 576-1146,bogdanovmoke@rambler.ru,"г. Елабуга, бул. 60 лет Октября, д. 5, 070955",1987-05-23,2016-07-14,F,NaN,75232,NaN,NaN
2,3,Валентина Игоревна Фомина,"к. Усть-Катав, бул. Восточный, д. 292 стр. 3, ...",+7 (920) 068-58-36,mjasnikovvalentin@mail.ru,"г. Сосновый Бор, алл. Лесхозная, д. 146, 139446",1974-06-15,2017-10-18,F,NaN,88813,NaN,NaN
3,4,тов. Воронцова Фёкла Федоровна,"клх Новая Игирма, алл. Пограничная, д. 6/6, 73...",8 334 398 3828,martin85@rambler.ru,"клх Котельнич, бул. Серафимовича, д. 97, 753076",1979-10-19,2012-08-25,F,NaN,55167,NaN,NaN
4,5,Филипп Дмитриевич Воронцов,"к. Ребриха, пр. Свободы, д. 54, 466509",8 (296) 455-0453,jan_07@rambler.ru,"клх Карабудахкент, ул. Абрикосовая, д. 3 к. 19...",1986-08-14,2017-03-15,M,NaN,85106,NaN,NaN


In [25]:
categories_df.head()

,id,name,description,mcc-code
0,1,Каршеринг,Краткосрочная аренда авто с оплатой по минутам...,"7512, 4121"
1,2,Супермаркеты,Покупки в супермаркетах и продуктовых магазинах,"5297, 5298, 5300, 5411, 5412, 5422, 5441, 5451..."
2,3,Такси,Услуги такси (каршеринг не входит в данную кат...,4121
3,4,Музыка,Покупки в магазинах музыки и музыкальных инстр...,"5733, 5735"
4,5,Фастфуд,Покупки в ресторанах быстрого питания,5814


### Предобработка данных

На этом этапе производится базовая очистка и приведение данных к корректным форматам перед загрузкой в базу SQLite.  
Основные задачи предобработки:

- проверка типов данных и преобразование текстовых значений в datetime, числовые и категориальные типы;
- выявление и обработка пропусков в ключевых полях;
- приведение колонок к единообразному формату для последующей записи в БД;
- первичная валидация структуры таблиц (соответствие дат, корректность идентификаторов, отсутствие дубликатов).

Такая предварительная обработка позволяет избежать ошибок при создании SQL-таблиц и гарантирует корректную работу аналитических запросов в дальнейшем.

In [ ]:
# приводим данные к формату datetime
clients_df["birthdate"] = pd.to_datetime(clients_df["birthdate"], errors="coerce")
clients_df["registration_date"] = pd.to_datetime(clients_df["registration_date"], errors="coerce")
subscriptions_df["date_start"] = pd.to_datetime(subscriptions_df["date_start"], errors="coerce")
subscriptions_df["date_end"] = pd.to_datetime(subscriptions_df["date_end"], errors="coerce")
transactions_df["date"] = pd.to_datetime(transactions_df["date"], errors="coerce")

In [33]:
# функция для удобного вывода информации по DataFrame
def inspect_df(df, name):
    print(f"{name}")
    print("Размер:", df.shape)
    # print("Типы данных:")
    # print(df.dtypes)
    print("Общее количество пропусков:", df.isna().sum().sum())

# проверяем все загруженные таблицы
inspect_df(categories_df, "categories_df")

categories_df
Размер: (29, 4)
Общее количество пропусков: 2


In [34]:
inspect_df(clients_df, "clients_df")

clients_df
Размер: (1000, 13)
Общее количество пропусков: 1896


In [35]:
inspect_df(subscriptions_df, "subscriptions_df")

subscriptions_df
Размер: (1308, 7)
Общее количество пропусков: 1531


In [36]:
inspect_df(transactions_df, "transactions_df")

transactions_df
Размер: (929135, 8)
Общее количество пропусков: 284484


### Создание БД
to do описать 

In [19]:
DB_PATH = "bank.db"

# для чистоты шага пересоздаём базу
if Path(DB_PATH).exists():
    Path(DB_PATH).unlink()

# создаем объект подключения к БД
conn = sqlite3.connect(DB_PATH)

# создаем таблицы, используя pandas
categories_df.to_sql("categories", conn, index=False)
clients_df.to_sql("clients", conn, index=False)
subscriptions_df.to_sql("subscriptions", conn, index=False)
transactions_df.to_sql("transactions", conn, index=False)

conn.close()

print(" SQLite база создана:", DB_PATH)

 SQLite база создана: bank.db


In [20]:
conn = sqlite3.connect(DB_PATH)

# выполняем SQL запрос к каждой таблице
for table in ["categories", "clients", "subscriptions", "transactions"]:
    n = pd.read_sql(f"SELECT COUNT(*) AS n FROM {table}", conn)["n"][0]
    print(table, n)

conn.close()

categories 29
clients 1000
subscriptions 1308
transactions 929135


### Вывод
to do описать, что пандас удобен, можно ипспользовать встроенные функции для создания таблиц, подключения к бд и т.д